# ETL Proyecto individual MLOps
Se presentan datos en formato json, procederemos a extraerlos y convertirlos en DataFrames para transformar los datos y posteriormente ser guardados en formato csv y parquet.

In [2]:
import pandas as pd
import json
import ast
import gzip
import pyarrow as pa
import pyarrow.parquet as pq

### ETL games.json.gz

In [6]:
filas = []

with gzip.open('steam_games.json.gz', 'rb') as archivo:
    for linea in archivo:
        try:
            objeto_json = json.loads(linea)
            filas.append(objeto_json)
        except json.JSONDecodeError:
            print(f'Error de formato JSON en la línea: {linea}')

#Convertir la lista de objetos JSON en un DataFrame.
df_games = pd.DataFrame(filas)

In [7]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eliminamos valores nulos

In [4]:
df_games.dropna(subset=['id'], inplace=True)

Eliminamos columnas irrelevantes y reseteamos indices

In [5]:
# Elimina columnas irrelevantes.
df_games.drop(['publisher','title', 'url', 'reviews_url', 'early_access', 'specs'], axis=1, inplace = True)

# Resetea indices. 
df_games.reset_index(drop=True, inplace=True)

In [6]:
df_games.head(3)

,genres,app_name,release_date,tags,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
4,NaN,Log Challenge,NaN,"[Action, Indie, Casual, Sports]",2.99,773570,NaN
5,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540,Trickjump Games Ltd
6,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",9.99,774276,Poppermost Productions
7,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Pro Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",18.99,774277,Poppermost Productions
8,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",29.99,774278,Poppermost Productions
9,"[Casual, Indie, Racing, Simulation]",Race,2018-01-04,"[Indie, Casual, Simulation, Racing]",NaN,768800,RewindApp


Extraemos el año de la columna release_date

In [ ]:
# Extrae el año de la columna release_date y lo almacena en la columna año_lanzamiento.
df_games['año_lanzamiento'] = df_games['release_date'].str.extract(r'(\d{4})')

# Elimina la columna release_date.
df_games.drop(columns=['release_date'],inplace=True)

In [18]:
# Verifica los nulos 
df_games.isnull().sum()

genres             0
app_name           0
tags               0
price              0
id                 0
developer          0
año_lanzamiento    0
dtype: int64

Reemplazamos los nulos por 0

In [19]:
df_games.fillna('0',inplace=True)
df_games.isnull().sum()

Renombramos la columna id por item_id y cambiamos el tipo de dato de la columna año de lanzamiento





In [21]:
# Renombra columna id por item_id.
df_games = df_games.rename(columns={'id': 'item_id'})

# Cambia el tipo de dato a int de la columna año de lanzamiento.
df_games['año_lanzamiento'] = df_games['año_lanzamiento'].astype(int)

Verificar si la columna precio contiene valores de tipo str

In [22]:
string_prices = df_games[df_games['price'].apply(lambda x: isinstance(x, str))]
print(string_prices['price'].value_counts())

price
0                                1377
Free                              905
Free to Play                      520
Free To Play                      462
Free Mod                            4
Free Demo                           3
Play for Free!                      2
Third-party                         2
Play Now                            2
Starting at $499.00                 1
Free Movie                          1
Free to Try                         1
Starting at $449.00                 1
Install Theme                       1
Play the Demo                       1
Free HITMAN™ Holiday Pack           1
Play WARMACHINE: Tactics Demo       1
Install Now                         1
Free to Use                         1
Name: count, dtype: int64


Convertir los valores de la columna 'price' que son strings a cero



In [23]:
df_games.loc[df_games['price'].apply(lambda x: isinstance(x, str)), 'price'] = 0

Guardar el dataframe en formato csv

In [24]:
games = df_games.to_csv('steam_games.csv',index=False)

In [25]:
# Lee el archivo CSV en un DataFrame de pandas.
games = pd.read_csv('steam_games.csv')

# Convierte el DataFrame de pandas a una tabla de PyArrow.
table = pa.Table.from_pandas(games)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, 'steam_games.parquet')

### ETL users_items.json.gz

In [26]:
users_items = []

with gzip.open('users_items.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            user_item_line = ast.literal_eval(line)
            users_items.append(user_item_line)
        except (SyntaxError, ValueError) as e:
            print(f'Error evaluating line as literal: {e}')

df_items = pd.DataFrame(users_items)
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [27]:
df_items = pd.DataFrame(users_items)
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Creamos una nueva fila para cada elemento de la lista de la columna items

In [28]:
df_items = df_items.explode('items').reset_index()
df_items = df_items.drop(columns='index')

In [29]:
df_items = pd.concat([df_items, pd.json_normalize(df_items['items'])], axis=1)
df_items.head()

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force,0.0,0.0


Eliminamos columnas irrelevantes

In [30]:
df_items.drop(['user_url','items'], axis=1, inplace=True)

In [31]:
# Verificamos los nulos
df_items.isnull().sum()

user_id                 0
items_count             0
steam_id                0
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

Eliminamos los nulos



In [32]:
df_items = df_items.dropna()
df_items.isnull().sum()

user_id             0
items_count         0
steam_id            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [33]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   item_id           object 
 4   item_name         object 
 5   playtime_forever  float64
 6   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 314.5+ MB


In [34]:
len(df_items)

5153209

In [35]:
#convertimos a csv
items= df_items.to_csv('users_items.csv',index=False) 

In [36]:
items= pd.read_csv('users_items.csv')

Reemplazamos las '' por vacío y cambiamos el tipo de dato de la columna items_id

In [37]:
items['items_id'] = items['item_id'].astype(int)

In [38]:
# Leer el archivo CSV en un DataFrame de pandas
items = pd.read_csv('users_items.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(items)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, './data/users_items.parquet')

In [39]:
# Leer el archivo Parquet en una tabla de PyArrow
table = pq.read_table('./data/users_items.parquet')

# Convertir la tabla de PyArrow a un DataFrame de pandas
items_parquet = table.to_pandas()

# Imprimir el DataFrame
items_parquet

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...
5153204,76561198329548331,7,76561198329548331,346330,BrainBread 2,0.0,0.0
5153205,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0,0.0
5153206,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5153207,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0


### ETL user_reviews.json.gz

In [40]:
users_reviews = []

with gzip.open('user_reviews.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            user_item = ast.literal_eval(line)
            users_reviews.append(user_item)
        except (SyntaxError, ValueError) as e:
            print(f'Error evaluating line as literal: {e}')

In [41]:
df_reviews = pd.DataFrame(users_reviews)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Hacemos explode en la columa reviews

In [42]:
df_reviews = df_reviews.explode('reviews').reset_index()
df_reviews = df_reviews.drop(columns='index')
df_reviews = pd.concat([df_reviews, pd.json_normalize(df_reviews['reviews'])], axis=1)
df_reviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Observamos los nombres de las columnas su tipo de dato y los no nulos.



In [43]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


In [44]:
nulos3 = df_reviews.isna().sum()
nulos3

user_id         0
user_url        0
reviews        28
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

Utilizar una expresión regular para extraer el año


In [45]:
df_reviews['año'] = df_reviews['posted'].str.extract(r'(\d{4})')
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,año
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011


In [46]:
df_reviews['año'].isna().value_counts()

año
False    49186
True     10147
Name: count, dtype: int64

In [47]:
df_reviews.isnull().sum()

user_id            0
user_url           0
reviews           28
funny             28
posted            28
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
año            10147
dtype: int64

Eliminamos los valores vacíos de la columna año



In [48]:
df_reviews = df_reviews.dropna(subset=['año'])
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,año
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011


In [49]:
df_reviews.isnull().sum()

user_id        0
user_url       0
reviews        0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
año            0
dtype: int64

Realizamos análisis de sentimientos



In [52]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(review):
    
    if isinstance(review, str):
        sentiment = analyzer.polarity_scores(review)

        if sentiment['compound'] >= 0.05:
            return 2
        elif sentiment['compound'] <= -0.05:
            return 0
        else:
            return 1
    else:
        return 1
        
#Aplica la función de análisis de sentimiento a la columna 'reviews' si existe al menos una reseña

df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)

In [53]:
df_reviews['sentiment_analysis'].value_counts()

sentiment_analysis
2    33422
1     9080
0     6684
Name: count, dtype: int64

In [54]:
df_reviews.drop(['reviews','user_url', 'funny', 'last_edited','posted','review'], axis=1, inplace=True)

In [55]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49186 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49186 non-null  object
 1   item_id             49186 non-null  object
 2   helpful             49186 non-null  object
 3   recommend           49186 non-null  object
 4   año                 49186 non-null  object
 5   sentiment_analysis  49186 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 2.6+ MB


In [56]:
#convertir a csv
reviews= df_reviews.to_csv('users_reviews.csv',index=False)

In [57]:
reviews=pd.read_csv('users_reviews.csv')

In [58]:
# Leer el archivo CSV en un DataFrame de pandas
reviews = pd.read_csv('users_reviews.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(reviews)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, './data/users_reviews.parquet')

In [ ]:
# Leer el archivo Parquet en una tabla de PyArrow
table = pq.read_table('./data/users_reviews.parquet')

# Convertir la tabla de PyArrow a un DataFrame de pandas
reviews_parquet = table.to_pandas()

# Imprimir el DataFrame
reviews_parquet

In [ ]:
# Leer el archivo CSV en un DataFrame de pandas
games_ml = pd.read_csv('steam_games_ml.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
table = pa.Table.from_pandas(games_ml)

# Escribir la tabla en un archivo Parquet
pq.write_table(table, './data/steam_games_ml.parquet')